# Install

In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


You should consider upgrading via the 'C:\Users\Deniz\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install stable-baselines3[extra] protobuf==3.20.*

You should consider upgrading via the 'C:\Users\Deniz\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install mss pydirectinput pytesseract gym

You should consider upgrading via the 'C:\Users\Deniz\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import numpy as np
from mss import mss
import pydirectinput
import pytesseract
from matplotlib import pyplot as plt
import time
from gym import Env
from gym.spaces import Box, Discrete, MultiDiscrete, Tuple
import copy
import random
import pygame
import threading
import os 
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker
import sys
from IPython import display

In [5]:
def list_to_array(list):
    updated_list = []
    for sublist in list:
        updated_sublist = []
        for i in sublist:
            updated_sublist.append(i+1)
        updated_list.append(updated_sublist)
    for sublist in updated_list:
        while len(sublist)<4:
            sublist.append(0)
    while len(updated_list) < 8:
        updated_list.append([-1,-1,-1,-1])
    return np.array(updated_list)

# Agent

In [7]:
class WaterSortAgent(Env):
    def __init__(self):
        self.level = 0
        self.tubes_number = 4
        self.new_game = True
        self.tube_colors = []
        self.initial_colors = []
        self.seed = 0
        self.selected = False
        self.tube_rects = []
        self.select_rect = 100
        self.win = False

        self.action_number = 0
        self.invalid_action_counter = 0
        self.last_three_observation = []

        #self.plot_scores = []
        #self.mean_score = []
        #self.total_score = 0
        #self.last_100_score = 0
        #plt.ion()
        
        self.observation_space = Box(low=-1, high=6, shape=(8, 4), dtype=int)
        self.action_space = MultiDiscrete([8, 8])
    
    def generate_start(self):
        #random.seed(self.seed)
        self.tubes_number = 8
        #self.tubes_number = random.randint(4, 8)
        tubes_colors = []
        available_colors = []
        for i in range(self.tubes_number):
            tubes_colors.append([])
            if i < self.tubes_number - 2:
                for j in range(4):
                    available_colors.append(i)
        for i in range(self.tubes_number - 2):
            for j in range(4):
                color = random.choice(available_colors)
                tubes_colors[i].append(color)
                available_colors.remove(color)
        self.level +=1
        self.seed +=1

        return tubes_colors

    def calc_move(self, colors, selected_rect, destination):
        if len(colors) > selected_rect and len(colors) > destination:
            chain = True
            color_on_top = 100
            length = 1
            color_to_move = 100
            if len(colors[selected_rect]) > 0:
                color_to_move = colors[selected_rect][-1]
                for i in range(1, len(colors[selected_rect])):
                    if chain:
                        if colors[selected_rect][-1 - i] == color_to_move:
                            length += 1
                        else:
                            chain = False
            if 4 > len(colors[destination]):
                if len(colors[destination]) == 0:
                    color_on_top = color_to_move
                else:
                    color_on_top = colors[destination][-1]
            if color_on_top == color_to_move:
                for i in range(length):
                    if len(colors[destination]) < 4:
                        if len(colors[selected_rect]) > 0:
                            colors[destination].append(color_on_top)
                            colors[selected_rect].pop(-1)

        return colors

    def check_victory(self, colors):
        won = True
        for i in range(len(colors)):
            if len(colors[i]) > 0:
                if len(colors[i]) != 4:
                    won = False
                else:
                    main_color = colors[i][-1]
                    for j in range(len(colors[i])):
                        if colors[i][j] != main_color:
                            won = False
        return won

    def get_done(self):
        done = False
        if self.invalid_action_counter == 1000:
            done = True
        return done

    def step(self, action):
        self.action_number += 1
        action_type = None
        reward = 0
        done = False
        info = {}

        if len(self.last_three_observation) == 0:
            self.last_three_observation.append(self.tube_colors)
        obs_list = copy.deepcopy(self.last_three_observation)

        self.tube_colors = self.calc_move(self.tube_colors, action[0], action[1])
        observation = copy.deepcopy(self.tube_colors)

        if self.check_victory(observation):
            reward = 10
            self.reward += reward
            done = True
            action_type = "WON"

        else:
            if len(obs_list) == 4:
                obs_list.pop(0)

            if sorted(obs_list[-1]) == sorted(observation):
                reward = -1
                action_type = "Not Changed"
                self.invalid_action_counter += 1

            elif len(obs_list) == 2 and sorted(obs_list[-2]) == sorted(observation):
                reward = -1
                action_type = "Iteration"
                self.invalid_action_counter += 1

            elif len(obs_list) == 3 and (sorted(obs_list[-2]) == sorted(observation) or sorted(obs_list[-3]) == sorted(observation)):
                reward = -1
                action_type = "Iteration"
                self.invalid_action_counter += 1

            elif self.check_fullcolor(observation) > self.check_fullcolor(obs_list[-1]):
                reward = 5
                action_type = "Fullcolor"
                obs_list.append(observation)

            else:
                reward = 1
                action_type = "Valid Action"
                obs_list.append(observation)

            self.reward += reward
        if done == False:
            done = self.get_done()
        self.last_three_observation = copy.deepcopy(obs_list)

        #print(action, action_type, "ActionCounter:", self.action_number, "Reward:", reward)
        #print(observation)
        #print()
        return list_to_array(observation), reward, done, info
    

    def reset(self):
        """if self.level != 0:
            self.last_100_score += self.reward
            if self.level % 100 == 99:
                self.plot_scores.append(self.last_100_score/100)
                self.total_score += self.last_100_score
                self.last_100_score = 0
                self.mean_score.append(self.total_score/(self.level))"""
        self.reward = 0
        observation = self.generate_start()
        while self.check_victory(observation) == True:
            observation = self.generate_start()
            self.level -= 1
        self.action_number = 0
        self.invalid_action_counter = 0
        self.last_three_observation = []
        self.tube_colors = observation

        #print(observation)
        #print()
        return list_to_array(observation)

    def check_fullcolor(self, obs):
        x = 0
        for i in obs:
            if len(i) == 4:
                if all(x == i[0] for x in i):
                    x += 1
        return x
                
    

In [8]:
import time
env = WaterSortAgent()
time.sleep(2)
for episode in range(10):
    obs = env.reset()
    done = False  
    total_reward   = 0
    while not done: 
        obs, reward,  done, info =  env.step(env.action_space.sample())
        total_reward  += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))   

Total Reward for episode 0 is -511
Total Reward for episode 1 is -712
Total Reward for episode 2 is -759
Total Reward for episode 3 is -976
Total Reward for episode 4 is -629
Total Reward for episode 5 is -369
Total Reward for episode 6 is -376
Total Reward for episode 7 is -984
Total Reward for episode 8 is -688
Total Reward for episode 9 is -392


In [31]:
model = A2C.load('train_first/level_4_deep28.zip') 

env = WaterSortAgent()
time.sleep(2)
for episode in range(10):
    obs = env.reset()
    done = False  
    total_reward   = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        total_reward  += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))    

Total Reward for episode 0 is -87
Total Reward for episode 1 is -91
Total Reward for episode 2 is -159
Total Reward for episode 3 is -72
Total Reward for episode 4 is -43
Total Reward for episode 5 is -49
Total Reward for episode 6 is -139
Total Reward for episode 7 is -63
Total Reward for episode 8 is -211
Total Reward for episode 9 is -44


# A2C Model

In [9]:
from stable_baselines3 import DQN, A2C, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.env_util import make_vec_env
import time

In [10]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

CHECKPOINT_DIR = "./train/"
LOG_DIR = "./logs/"

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [174]:
import time
start_time = time.time()


env = WaterSortAgent()
time.sleep(4)

model = A2C('MlpPolicy', env, tensorboard_log=LOG_DIR, verbose=1, device="cuda", n_steps=1000)
model.learn(total_timesteps=100000)


finish_time = time.time()
time = finish_time - start_time

hours = int(time // 3600)
minutes = int((time % 3600) // 60)
seconds = int(time % 60)

print("Time for learning: {} hours, {} minutes, {} seconds".format(hours, minutes, seconds))

KeyboardInterrupt: 

In [13]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv


env = WaterSortAgent()
#old_model = A2C.load('train_first/level_4_deep27.zip') 

num_envs = 100
vec_env = DummyVecEnv([lambda: WaterSortAgent() for _ in range(num_envs)])

custom_arch = dict(pi=[256, 256, 256], vf=[256, 256, 256])
model = A2C('MlpPolicy', vec_env, tensorboard_log=LOG_DIR, verbose=1, device="cuda", n_steps=100, policy_kwargs=dict(net_arch=custom_arch))
#model.set_parameters(old_model.get_parameters())

model.learn(total_timesteps=10000000)

Using cpu device
Logging to ./logs/A2C_119
------------------------------------
| time/                 |          |
|    fps                | 4517     |
|    iterations         | 100      |
|    time_elapsed       | 221      |
|    total_timesteps    | 1000000  |
| train/                |          |
|    entropy_loss       | -4.03    |
|    explained_variance | 2.86e-05 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -53.3    |
|    value_loss         | 410      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 4524     |
|    iterations         | 200      |
|    time_elapsed       | 442      |
|    total_timesteps    | 2000000  |
| train/                |          |
|    entropy_loss       | -3.79    |
|    explained_variance | 9.06e-06 |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | -22.2 

In [14]:
model.save('train_first/level_5') 